<a href="https://colab.research.google.com/github/JeisonTantachuco/CursoGEE/blob/main/module05/05_Ejercicio_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<!--COURSE_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://user-images.githubusercontent.com/16768318/73986808-75b3ca00-4936-11ea-90f1-3a6c352766ce.png" width=10% >
<img align="right" style="padding-left:10px;" src="https://user-images.githubusercontent.com/16768318/73986811-764c6080-4936-11ea-9653-a3eacc47caed.png" width=10% >

**Bienvenidos!** Este *colab notebook* es parte del curso [**Introduccion a Google Earth Engine con Python**](https://github.com/csaybar/EarthEngineMasterGIS) desarrollado por el equipo [**MasterGIS**](https://www.mastergis.com/). Obten mas informacion del curso en este [**enlace**](https://www.mastergis.com/product/google-earth-engine/). El contenido del curso esta disponible en [**GitHub**](https://github.com/csaybar/EarthEngineMasterGIS) bajo licencia [**MIT**](https://opensource.org/licenses/MIT).

### **Ejercicio N°02:**
 - Cree una funcion (o funciones) que simplifique el proceso de control de calidad de imagenes sentinel2. Esta funcion debe tener como argumentos:
   **clean_s2(init_date, last_date, roi, cloud_per, calendar, bands)**
   - **init_data** : La fecha de inicio
   - **last_date** : La fecha de fin.
   - **roi** : El ambito de estudio.
   - **cloud_per** : El porcentaje de nubes.
   - **calendar** : Los meses a considerar para la seleccion de imagenes.
   - **bands** : Las bandas de interes.

<img src="https://user-images.githubusercontent.com/16768318/73661182-8bed2c00-4691-11ea-8c5d-f96a0ad3e827.png" align="right" width = 60%/>

#### **1) Autenticar y inicializar GEE**

In [1]:
import ee
ee.Authenticate()
ee.Initialize(project = 'ee-jtantaroman')

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


#### **2) Carga nuestra funcion de mapeo**

In [2]:
#@title mapdisplay: Crea mapas interactivos usando folium
import folium
def mapdisplay(center, dicc, Tiles="OpensTreetMap",zoom_start=10):
    '''
    :param center: Center of the map (Latitude and Longitude).
    :param dicc: Earth Engine Geometries or Tiles dictionary
    :param Tiles: Mapbox Bright,Mapbox Control Room,Stamen Terrain,Stamen Toner,stamenwatercolor,cartodbpositron.
    :zoom_start: Initial zoom level for the map.
    :return: A folium.Map object.
    '''
    center = center[::-1]
    mapViz = folium.Map(location=center,tiles=Tiles, zoom_start=zoom_start)
    for k,v in dicc.items():
      if ee.image.Image in [type(x) for x in v.values()]:
        folium.TileLayer(
            tiles = v["tile_fetcher"].url_format,
            attr  = 'Google Earth Engine',
            overlay =True,
            name  = k
          ).add_to(mapViz)
      else:
        folium.GeoJson(
        data = v,
        name = k
          ).add_to(mapViz)
    mapViz.add_child(folium.LayerControl())
    return mapViz

####**3) Area de Estudio**

In [4]:
#@title piura
piura =  {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -80.70144653320312,
              -5.247546429206079
            ],
            [
              -80.59295654296875,
              -5.247546429206079
            ],
            [
              -80.59295654296875,
              -5.153521331735138
            ],
            [
              -80.70144653320312,
              -5.153521331735138
            ],
            [
              -80.70144653320312,
              -5.247546429206079
            ]
          ]
        ]
      }
    }
  ]
}

ee_piura = ee.Feature(piura['features'][0])
centroide = ee_piura.geometry().centroid().getInfo()['coordinates']

In [5]:
dicc = {'area_de_estudio':ee_piura.getInfo()}
mapdisplay(centroide, dicc, zoom_start= 10)

####**4) Funcion para crear ee.ImageCollection**`
En realidad este 512, deberia ser 2048 porque ese bit representa a 1x2^11

<center>
<img src="https://user-images.githubusercontent.com/16768318/72954392-1b403880-3d90-11ea-9956-15e607f00fcb.png" align="right" width = 60%/>
</center>




En la siguiente función se utiliza el bitwiseand donde esta operador trabaja de la sgte forma:
caso 1: 1 y 0 -> 0
caso 2: 0 y 1 -> 0
caso 3: 0 y 0 -> 0
caso 4: 1 y 1 -> 1

Es decir al performar esa operacion por ejemplo en un pixel que no tiene nube opaca:



*   01000000000  (opaque_cloud)
*   00001101001  (qa)
*   resulta:
*   00000000000


Como este valor es igual a 0, al compararlo si es igual a 0 nos dará True (1). Estos valores son los que queremos utilizar en la máscara.

Pero para el caso donde el pixel si sea una nube opaca:


*   01000000000  (opaque_cloud)
*   01001101001  (qa)
*   resulta:
*   01000000000


Este valor no es igual a 0 sino a 1*2^10, asi que al hacerle el eq(0) (compararlo si es igual a 0) nos dará False que es 0. Estos valores no se enmascarán y de esa forma se limpiarán.


De esa forma también se va a considerar las nubes cirrus donde finalmente se aplicará el .And





*   sin nube opaca? sin nube cirrus?
*   1             and   1        -> 1 ( sin nube opaca y sin nube cirrus)
*   0             and   1        -> 0 (solo sin nube opaca, pero si hay cirrus)
*   1             and   0        -> 0 (solo sin nube cirrus, pero si hay opaca
*   0             and   0        -> 0 (con nube opaca y cirrus)







In [6]:
def clean_s2(init_date, last_date, roi, cloud_per, calendar, bands):
  def maskS2clouds(image):
    qa = image.select('QA60')
    opaque_cloud = 1 << 10
    cirrus_cloud = 1 << 11
    mask = qa.bitwiseAnd(opaque_cloud).eq(0)\
             .And(qa.bitwiseAnd(cirrus_cloud).eq(0))
    clean_image = image.updateMask(mask).select(bands)
    return clean_image.copyProperties(image, ['system:time_start']) #luego de utilizar una funcion map las propiedades de entrada de la imagen se pierden por eso se utiliza copy properties
  ic_s2 = ee.ImageCollection("COPERNICUS/S2_SR")\
               .filterBounds(roi)\
               .filterDate(init_date, last_date)\
               .filter(ee.Filter.calendarRange(calendar[0], calendar[1],'month'))\
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_per))\
               .map(maskS2clouds)
  return ic_s2

#### **5) Funcion para obtener las fechas en Google Earth Engine**

In [13]:
from datetime import datetime as dt
def get_dates(ic):
  dates = ic.aggregate_array('system:time_start').getInfo()
  anonym = lambda x: dt.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S') #Se divide /1000 porque las fechas en python son en segundos y en GEE en milisegundos
  return list(map(anonym, dates))

#### **6) Funcion para monitorear las descargar automaticamente**

In [15]:
from ee.batch import Export
from datetime import datetime
import time

def ee_monitoring(ee_task):
  while ee_task.active():
    print('Sondeo de la tarea (id: {}).'.format(ee_task.id))
    time.sleep(5)

#### **7) Funcion para descargar automaticamente**

In [16]:
def ee_download_images(ic, region, prefix, folder, scale):
  nimages = ic.size().getInfo() #obteniendo el nro de imagenes
  ic_list = ic.toList(nimages)  #pasando el imagecollection a una lista de imagenes(todas las imagenes en una lista)
  for n_img in range(nimages):
    to_download = ee.Image(ic_list.get(n_img))
    timestamp = to_download.get('system:time_start').getInfo()
    dt_object = datetime.fromtimestamp(timestamp/1000).strftime('%Y_%m_%d_%H_%M_%S')
    task = Export.image.toDrive(
      image=to_download,
      description= prefix+dt_object,
      folder=folder,
      scale=scale,
      region = region
    )
    print('Descargando: '+prefix+dt_object+'.tif')
    task.start()
    ee_monitoring(task)

#### **8) Usa tus funciones para descargar masivamente los datos**

In [12]:
#@title piura
piura =  {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -80.70144653320312,
              -5.247546429206079
            ],
            [
              -80.59295654296875,
              -5.247546429206079
            ],
            [
              -80.59295654296875,
              -5.153521331735138
            ],
            [
              -80.70144653320312,
              -5.153521331735138
            ],
            [
              -80.70144653320312,
              -5.247546429206079
            ]
          ]
        ]
      }
    }
  ]
}

ee_piura = ee.Feature(piura['features'][0])
centroide = ee_piura.geometry().centroid().getInfo()['coordinates']

In [7]:
s2 = clean_s2(init_date = '2019-01-01',
              last_date = '2019-12-01',
              roi = ee_piura.geometry(),
              cloud_per = 1,
              calendar =  [4,10],
              bands =  ['B4','B3','B2'])

/usr/local/lib/python3.10/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


In [11]:
#Tenemos 14 imagenes pero si queremos saber las fechas de obtencion?
s2.size().getInfo()

14

In [14]:
get_dates(s2) #Utilizando la funcion get dates

['2019-04-10 15:54:38',
 '2019-04-12 15:44:46',
 '2019-04-30 15:54:40',
 '2019-06-09 15:54:39',
 '2019-07-11 15:44:50',
 '2019-07-16 15:44:46',
 '2019-07-21 15:44:50',
 '2019-07-31 15:44:49',
 '2019-08-13 15:54:43',
 '2019-08-28 15:54:37',
 '2019-09-07 15:54:35',
 '2019-09-12 15:54:40',
 '2019-09-14 15:44:40',
 '2019-10-14 15:44:44']

In [18]:
ee_download_images(ic = s2,
                   region =  ee_piura.geometry(),
                   prefix =  'PIURA_SENTINEL2_',
                   folder = 'MASTERGIS_GEE2',
                   scale =  10)

Descargando: PIURA_SENTINEL2_2019_04_10_15_54_38.tif
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Sondeo de la tarea (id: CMFYQ5X7B5ZRR3XPKTTWYBUA).
Descargando: PIURA_SENTINEL2_2019_04_12_15_44_46.tif
Sondeo de la tarea (id: RTA4UUTMPKD72PN7Y2Q4MCZZ).
Sondeo de la tarea (id: RTA4UUTMPKD72PN7Y2Q4MCZZ).
Sondeo de la tarea (id: RTA4UUTMPKD72PN7Y2Q4MCZZ).
Sondeo de la tarea (id: RTA4UUTMPKD72PN7Y2Q4MCZZ).
Sondeo de la tarea (id: RTA4UUTMPKD72PN7Y2Q4MCZZ).
Sondeo de la tarea (id: RTA4UUTMPKD72PN7Y2Q4MCZZ).
Sondeo de la tarea (id: RTA

### **¿Dudas con este Jupyer-Notebook?**

Estaremos felices de ayudarte!. Create una cuenta Github si es que no la tienes, luego detalla tu problema ampliamente en: https://github.com/csaybar/EarthEngineMasterGIS/issues

**Tienes que dar clic en el boton verde!**

<center>
<img src="https://user-images.githubusercontent.com/16768318/79680748-d5511000-81d8-11ea-9f89-44bd010adf69.png" width = 70%>
</center>